# ATMO 5331 - Homework 2 - Fall 2021
## Due 16 Sep, 2021

When doing this homework, remember that you have two jobs:
1. Make it work.
2. Clean it up so that I can understand what you've done. If you think I might not undersand, document it with a comment or a function docstring.

You should present your work with a clear logical progression. If that seems like a hassle, remember that in doing so you are practicing skills that are expected in your thesis and journal publications.

You may work alone or in pairs. I will not be adjudicating relative level of effort in group work, so you are responsible for ensuring there is an even contribution by your partner.

**Question 1**

Grab the [WGS84 implementation manual](https://www.icao.int/safety/pbn/documentation/eurocontrol/eurocontrol%20wgs%2084%20implementation%20manual.pdf), and implement a translation from geodetic latitude, longitude, and altitude (referenced to the WGS84 ellisoid) to the local XYZ cartesian system used for WGS84.

Use part 1 of Helmert's formula on p. 81 (Appendix E), and refer to Fig. B-6 in Appendix B (p. 70) for information about the coordinate system notation.

Careful with degrees and radians.

Compare your results to what you get when using the `proj4` library. This library has its origins in public domain code written by the USGS, and is used in many open source packages, including the QGIS system. For easy use of the `proj4` library, you can use the [helper routines in coordinateSystems.py](https://github.com/deeplycloudy/lmatools/blob/minvaluegrids/lmatools/coordinateSystems.py) from my `lmatools` package. Just copy that file into the same directory as this notebook.

You set up a coordinate system transform object as shown below. It defaults to a WGS84 ellipsoid, so we don't have to specify that. Once the coordinate system object `geo` has been created, you can reuse it withouth calling `GeographicSystem()` again. It accepts arrays of data.
```
from coordinateSystems import GeographicSystem 
geo = GeographicSystem() 
X, Y, Z = geo.toECEF(lon, lat, alt) # Use degrees
```

For your dataset, please use: 
```
import numpy as np
lat = np.array([  33.5,   1.0,   0.0,   0.0,   0.0,  10.0, -10.0]) 
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0]) 
alt = np.zeros_like(lat)
```

Demonstrate that your ECEF conversion equals that provided by the coordinate system library.


In [4]:
#Import necessary modules
import numpy as np
import pyproj as proj4

#Create an array of latitude, longitude, and altitude values
lat = np.array([  33.5,   1.0,   0.0,   0.0,   0.0,  10.0, -10.0]) 
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0]) 
alt = np.zeros_like(lat)

#Values of the semi-major axis of the ellipsoid (a) and the 
#flattening of the ellipsoid (f) for the WGS 1984 from page 71
a = 6378137
f = 1/298.257223563

#Calculate values given the equations on page 81
e2 = f*(2-f)  #
v = a/np.sqrt((1-(e2*(np.sin(np.deg2rad(lat)))**2)))
X1 = (v + alt)*np.cos(np.deg2rad(lat))*np.cos(np.deg2rad(lon))
Y1 = (v + alt)*np.cos(np.deg2rad(lat))*np.sin(np.deg2rad(lon))
Z1 = (v*(1-e2)+alt)*np.sin(np.deg2rad(lon))

#Print Variables
print(X1, Y1, Z1)

[-1061448.75418035  1650533.58831094   555891.26758132  2695517.17208404
  1650783.32787306  1625868.32721344  1625868.32721344] [-5217187.30723133 -6159875.21117539 -6353866.26310279 -5780555.22988658
 -6160807.25190988 -6067823.20357756 -6067823.20357756] [-6214593.57597707 -6119570.70607927 -6311331.06793104 -5741857.99662372
 -6119564.46711958 -6120182.20907031 -6120182.20907031]


In [5]:
#Use the coordinateSystems.py file to calculate X, Y, and Z and compare them to the ones we 
#calculated above to show they are the same
from coordinateSystems import GeographicSystem 
geo = GeographicSystem() 
X2, Y2, Z2 = geo.toECEF(lon, lat, alt) # Use degrees

#Print Variables and compare
print(X2, Y2, Z2)

# We see that X1, Y1, Z1 are equal to X2, Y2, Z2

[-1061448.75418035  1650533.58831094   555891.26758132  2695517.17208404
  1650783.32787306  1625868.32721344  1625868.32721344] [-5217187.30723133 -6159875.21117539 -6353866.26310279 -5780555.22988658
 -6160807.25190988 -6067823.20357756 -6067823.20357756] [ 3500334.28802236   110568.77482457        0.                0.
        0.          1100248.54773536 -1100248.54773536]


**Question 2.**

Using the `TangentPlaneCartesianSystem` class, convert the geodetic coordinates to local $(x, y, z)$. Create three tangent planes:

- A tangent plane centered at the MCOM building on the TTU campus, at the height of the ground at that location.

- A tangent plane centered at the MCOM building on the TTU campus, at the ellipsoid.

- A tangent plane directly below the GOES-East satellite at -75.0 degrees longitude.

Use `TangentPlaneCartesianSystem?` in the notebook to learn about the arguments accepted by the projection class. It has the same `.toECEF` and `.fromECEF` methods as the `GeographicSystem`.

Transform the geodetic dataset from the first problem into coordiantes with respect to each tangent plane.

You do not need to use the NAD83 locations of MCOM. They are only there to show my work on how I obtained the vertical position of MCOM in WGS84.

**a.** Using only the GOES-East tangent plane and the transformed geodetic dataset, show that the WGS84 earth shape is not spherical.

**b.** What is a rough, easily memorable rule of thumb for the number of kilometers per degree latitude?

**c.** Print out the tangent plane $(x,y,z)$ of the zeroth data point (it is a bit east of Lubbock). Explain why the differences in the coordinates of the two MCOM tangent planes make sense.

**d.** Imagine that there was no terrain, so that a radar located at MCOM was precisely on the WGS84 ellipsoid. If that radar were to scan toward the zeroth position at 0° elevation angle (assume no atmospheric refraction), how high above the ground would the beam be? Is this disance measured perpendicular to the ellipsoid or perpendicular to the tangent plane?

**e.** Transform your coordiantes back to ECEF from each tangent plane and show they're equal.

In [18]:
from coordinateSystems import TangentPlaneCartesianSystem

# From USGS Elevation point query service
# https://nationalmap.gov/epqs/
# NAD83 lon, lat and NAVD88 vertical
mcom_lat_nad83, mcom_lon_nad83 = 33.581857, -101.880360 # NAD83
mcom_alt_nad83 = 983.15

# Using https://vdatum.noaa.gov/vdatumweb/, convert the above to "WGS84 G1674 (Use ITRF2008)"
mcom_lon, mcom_lat = -101.8803718553, 33.5818617015
mcom_alt = 957.179

# The altitude difference is about the height of the geoid at this location.

#A tangent plane centered at the MCOM building on the TTU campus, at the height of the ground at that location
MCOM_loc = TangentPlaneCartesianSystem(mcom_lat, mcom_lon, mcom_alt)
loc_x, loc_y, loc_z = MCOM_loc.fromECEF(X2, Y2, Z2)

print(loc_x, loc_y, loc_z)
print('')
print('')

#A tangent plane centered at the MCOM building on the TTU campus, at the ellipsoid.
MCOM_loc_ellip = TangentPlaneCartesianSystem(mcom_lat, mcom_lon, mcom_alt-mcom_alt_nad83)
loc_x_ellip, loc_y_ellip, loc_z_ellip = MCOM_loc_ellip.fromECEF(X2, Y2, Z2)

print(loc_x_ellip, loc_y_ellip, loc_z_ellip)

print('')
print('')

MCOM_loc_GOES = TangentPlaneCartesianSystem(0, -75, 0)
loc_x_GOES, loc_y_GOES, loc_z_GOES = MCOM_loc_GOES.fromECEF(X2, Y2, Z2)

print(loc_x_GOES, loc_y_GOES, loc_z_GOES)

[  35344.86072647 2883305.54875458 1852047.67524291 3827814.92297788
 2883741.81704395 2840217.97714332 2840217.9771423 ] [   -9014.72096474 -3034455.40531721 -3356222.20176266 -2802257.46562071
 -3127045.90856091 -2162939.56619643 -3996165.74781827] [-1.06140674e+03 -1.57263626e+06 -1.28790284e+06 -2.12225973e+06
 -1.63307792e+06 -1.09603202e+06 -2.31318823e+06]


[  35344.86072647 2883305.54875458 1852047.67524291 3827814.92297788
 2883741.81704395 2840217.97714332 2840217.97714229] [   -9014.72096474 -3034455.4053166  -3356222.20176216 -2802257.46561989
 -3127045.90856028 -2162939.566196   -3996165.74781737] [-7.82567358e+01 -1.57165311e+06 -1.28691969e+06 -2.12127658e+06
 -1.63209477e+06 -1.09504887e+06 -2.31220508e+06]


[-2.37558820e+06 -1.40916044e-06 -1.10755187e+06  1.10755187e+06
  0.00000000e+00 -1.40418937e-05  1.39987254e-05] [ 3.50033429e+06  1.10568775e+05 -1.41136132e-05  1.41131363e-05
  0.00000000e+00  1.10024855e+06 -1.10024855e+06] [-1.61344419e+06 -9.64919572e+02 -

In [12]:
# Part A


In [ ]:
# Part B

In [ ]:
# Part C

In [ ]:
# Part D

In [22]:
# Part E

MCOM_ECEF_x, MCOM_ECEF_y, MCOM_ECEF_z = TangentPlaneCartesianSystem.toECEF(MCOM_loc,loc_x, loc_y, loc_z)

print(MCOM_ECEF_x, MCOM_ECEF_y, MCOM_ECEF_z)
print('')
print('')

#A tangent plane centered at the MCOM building on the TTU campus, at the ellipsoid.
ellip_ECEF_x, ellip_ECEF_y, ellip_ECEF_z = TangentPlaneCartesianSystem.toECEF(MCOM_loc_ellip, loc_x_ellip, loc_y_ellip, loc_z_ellip)

print(ellip_ECEF_x, ellip_ECEF_y, ellip_ECEF_z)

print('')
print('')

GOES_ECEF_x, GOES_ECEF_y, GOES_ECEF_z = TangentPlaneCartesianSystem.toECEF(MCOM_loc_GOES, loc_x_GOES, loc_y_GOES, loc_z_GOES)

print(GOES_ECEF_x, GOES_ECEF_y, GOES_ECEF_z)

[-1061448.75418035  1650533.58831094   555891.26758132  2695517.17208404
  1650783.32787306  1625868.32721344  1625868.32721344] [-5217187.30723133 -6159875.21117538 -6353866.26310278 -5780555.22988657
 -6160807.25190987 -6067823.20357756 -6067823.20357756] [ 3.50033429e+06  1.10568775e+05 -9.77888703e-09 -1.07102096e-08
 -9.77888703e-09  1.10024855e+06 -1.10024855e+06]


[-1061448.75418035  1650533.58831094   555891.26758132  2695517.17208404
  1650783.32787306  1625868.32721344  1625868.32721344] [-5217187.30723133 -6159875.21117539 -6353866.26310279 -5780555.22988658
 -6160807.25190988 -6067823.20357756 -6067823.20357756] [ 3.50033429e+06  1.10568775e+05 -2.79396772e-09 -3.25962901e-09
 -2.79396772e-09  1.10024855e+06 -1.10024855e+06]


[-1061448.75418035  1650533.58831094   555891.26758132  2695517.17208404
  1650783.32787306  1625868.32721344  1625868.32721343] [-5217187.30723134 -6159875.21117539 -6353866.26310279 -5780555.22988658
 -6160807.25190988 -6067823.20357757 -6067823.20

**Question 3** Use the `GeostationaryFixedGridSystem` to define two more coordiante transformations for the GOES-East and GOES-West locations at -75.0 and -135.0 degrees longitude.

Use `GeostationaryFixedGridSystem?` in the notebook to learn about the arguments accepted by the projection class.

Convert the dataset to fixed grid coordinates.

For more on fixed grid coordiantes, you can read the [GOES-R L1b product users' guide](https://www.goes-r.gov/resources/docs.html) and [my own description of GOES fixed grid coordiantes](https://github.com/deeplycloudy/glmtools/blob/master/docs/fixedgridguide.md), a.k.a. [the geostationary projection](https://proj.org/operations/projections/geos.html).



In [ ]:
from coordinateSystems import GeostationaryFixedGridSystem


**Question 4.** Make a plot of the east and north coordinate data from the lon/lat, 3 tangent plane, and two GOES fixed grid projections, for a total of 6 plots.

Transform and plot three more locations (your hometown, your undergraduate institution's location, and the farthest you've been from home).

Does everything make sense? If not, what do you observe? Offer a plausible explanation for what might have happened.

Label the coordinates with the altitude of the point in that coordinate reference system.

In [ ]:
loc_lon = 
loc_lat = 
loc_alt = 


%matplotlib notebook
import matplotlib.pyplot as plt
n_rows, n_cols = 3, 2
fig, axes = plt.subplots(n_rows, n_cols, figsize=(8,8))

axes[0,0].plot
axes[0,0].set_xlabel('Longitude')
axes[0,0].set_ylabel('Latitude')
axes[0,0].set_title('WGS84')
axes[0,0].plot(lon, lat, marker='.', linestyle='none')
axes[0,0].plot(loc_lon, loc_lat, marker='.', linestyle='none')
for tlon, tlat, tlabel in zip(lon, lat, alt):
    axes[0,0].text(tlon, tlat, tlabel)
for tlon, tlat, tlabel in zip(loc_lon, loc_lat, loc_alt):
    axes[0,0].text(tlon, tlat, tlabel)


# Make the other five panels here
    
    
fig.tight_layout()

In [ ]:
# BONUS! Make a 3D plot of all locations. 
# Try to imagine the curved earth surface on which they reside.
# This part is not graded, but might be useful to you.

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

ax.plot(X, Y, Z, marker='o', linestyle='none') # original locations
ax.plot(locX, locY, locZ, marker='o', linestyle='none') # personal locations
ax.set_xlabel('ECEF X (m)')
ax.set_ylabel('ECEF Y (m)')
ax.set_zlabel('ECEF Z (m)')
# ax.set_aspect('equal')

**5.** Using the arrays you created in the previous assignment, create a `pcolormesh` plot of the data in geostationary coordinates from both the GOES East and GOES West positions. (15 pts.)

**6.** One thing we didn't do in the previous assignment was plot in a "traditional" map projection. We'll do that now with the Azimuthal Equidistant and Gnomonic projections, centered on MCOM, as defined below. The MapProjection class has the same to/from ECEF methods, and coordinates returned are in meters relative to the center point. (15 pts.)

If you're curious, you can peruse [the full list of projections](https://proj.org/operations/projections/index.html) to see how to define others.

Create a plot of the same data in each map projection, and set the axis limits to +/- 1600 km. Do you notice any differences in the two projections?

In [ ]:
aeqd = MapProjection(projection='aeqd', lon_0=mcom_lon, lat_0=mcom_lat)
gnom = MapProjection(projection='gnom', lon_0=mcom_lon, lat_0=mcom_lat)

